In [2]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)

In [3]:
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [12]:
def get_data():
    def norm(x):
        std = np.std(x ,ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/1 Джарвис' # positive class

    PATHS = li.util.find_files(PATH_0)[:1000] + li.util.find_files(PATH_1)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    SOUNDS = []
    for path in PATHS:
        SOUNDS.append(Audio(path))
        print('Loading {:.1f}%'.format(len(SOUNDS) / N * 100))
        ipd.clear_output(wait=True)
    DATA = []
    LABELS = []

    for sound in SOUNDS:
        DATA.append(norm(sound.e_parts))
        LABELS.append(sound.label)

    return (np.array(DATA), np.array(LABELS))

In [13]:
data, labels = get_data()

Loading 100.0%


In [88]:
test_data, test_labels = data[8000:], labels[8000:]
data, labels = data[:8000], labels[:8000]

In [6]:
data.shape, labels.shape

((1081, 20), (1081,))

In [48]:
BATCH_SIZE = 1081

dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 20), (None,)), types: (tf.float64, tf.int32)>

In [14]:
BATCH_SIZE = 1081
test_data = np.reshape(data, [1081, 1, 20])

dataset = tf.data.Dataset.from_tensor_slices((test_data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 1, 20), (None,)), types: (tf.float64, tf.int32)>

In [188]:
data = np.reshape(data, (10070, 20))

In [211]:
labels = np.reshape(labels, (10070,))

In [44]:
np.shape(data)

(1070, 20)

<h1>Создание модели нейронной сети</h1>

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 20,)),
    #tf.keras.layers.GRU(20, kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(100, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(50, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(25, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(12, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    tf.keras.layers.Dense(2, activation='softmax')
], name='Jarvis_Dense')

model.summary()

Model: "Jarvis_Dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1, 100)            2100      
_________________________________________________________________
dense_1 (Dense)              (None, 1, 50)             5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1, 25)             1275      
_________________________________________________________________
dense_3 (Dense)              (None, 1, 12)             312       
_________________________________________________________________
dense_4 (Dense)              (None, 1, 2)              26        
Total params: 8,763
Trainable params: 8,763
Non-trainable params: 0
_________________________________________________________________


<h1>Обучение</h1>

In [9]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=10000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

CategoricalCrossentropy): 0.000, Recall: 0.983047604560852, Precision: 0.9956768751144409
Epoch 8290 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.98306804895401, Precision: 0.9956821799278259
Epoch 8300 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9830884337425232, Precision: 0.9956874251365662
Epoch 8310 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9831088185310364, Precision: 0.9956926703453064
Epoch 8320 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.98312908411026, Precision: 0.9956979155540466
Epoch 8330 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9831493496894836, Precision: 0.9957031607627869
Epoch 8340 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9831695556640625, Precision: 0.9957083463668823
Epoch 8350 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9831897020339966, Precision: 0.9957135915756226
Epoch 8360 -- Loss(SparseCategoricalCrossentropy): 0.000, Recall: 0.9832097887992859, Precision: 0.995

In [27]:
model.save('models\{}-v2.0.h5'.format(model.name))

In [8]:
model = tf.keras.models.load_model(r'models\Mira-v1.0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [15]:
predictions = tft.predict_classes(model=model, features=test_data)

In [16]:
n = 0

for i in range(len(labels)):
    n += int(labels[i] == predictions[i])

print(n, n / len(labels) * 100)

1081 100.0


In [17]:
predictions[60:70]

<tf.Tensor: shape=(10, 1), dtype=int64, numpy=
array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0]], dtype=int64)>

In [25]:
predictions[:15], labels[:15]

(<tf.Tensor: shape=(15, 1), dtype=int64, numpy=
 array([[0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [0],
        [1]], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]))

In [23]:
model(data[:15])

<tf.Tensor: shape=(15, 2), dtype=float32, numpy=
array([[1.0000000e+00, 2.5234957e-35],
       [1.0000000e+00, 7.1878402e-12],
       [1.0000000e+00, 6.2893484e-34],
       [1.0000000e+00, 3.4782343e-15],
       [1.0000000e+00, 1.2099237e-34],
       [1.0000000e+00, 4.2744404e-35],
       [1.0000000e+00, 1.6721829e-37],
       [1.0000000e+00, 1.9608733e-14],
       [1.0000000e+00, 8.6516753e-30],
       [1.0000000e+00, 2.2747894e-31],
       [1.0000000e+00, 2.7213112e-30],
       [1.0000000e+00, 6.4561496e-13],
       [1.0000000e+00, 2.5592202e-12],
       [1.0000000e+00, 3.7977355e-13],
       [3.8644362e-08, 1.0000000e+00]], dtype=float32)>

In [102]:
labels[60:70]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [134]:
model(np.reshape(data[0], (1, 1, 20)))

<tf.Tensor: shape=(1, 1, 2), dtype=float32, numpy=array([[[9.9980623e-01, 1.9373836e-04]]], dtype=float32)>

In [88]:
test_data[0].shape

(1, 20)

In [89]:
data[0].shape

(20,)

In [107]:
test_model = tf.keras.models.clone_model(model)

In [108]:
test_model(test_data[50:70])

<tf.Tensor: shape=(20, 2), dtype=float32, numpy=
array([[0.3673443 , 0.6326557 ],
       [0.43950397, 0.56049603],
       [0.6150811 , 0.38491893],
       [0.60334784, 0.39665216],
       [0.5920435 , 0.40795645],
       [0.71854764, 0.2814524 ],
       [0.6459553 , 0.3540447 ],
       [0.60502183, 0.3949782 ],
       [0.37577388, 0.62422615],
       [0.61354023, 0.3864598 ],
       [0.36829096, 0.63170904],
       [0.5177707 , 0.48222935],
       [0.59354055, 0.40645948],
       [0.6889333 , 0.31106666],
       [0.42778772, 0.5722123 ],
       [0.4162618 , 0.5837382 ],
       [0.49124718, 0.5087528 ],
       [0.58858407, 0.41141596],
       [0.27925968, 0.72074026],
       [0.57293254, 0.42706743]], dtype=float32)>

'Mira GRU'